# Training MLP model on insurance claims data

In [ ]:
from dydx.layers import Linear, Loss
from dydx.model import Model
from dydx.dataset import DataLoader, Dataset


l1 = Linear( random=True, dims=(8, 128))
l2 = Linear( random=True,dims=(128, 256))
l3 = Linear( random=True, dims=(256, 128))
l4 = Linear( random=True,dims=(128, 32))
l5 = Linear( activation=False, random=True, dims=(32,1)
)

		
names = ["l1","l2","l3","l4", "l5"]		
layers =[l1,l2,l3,l4, l5]
ls = dict(zip(names,layers))

model = Model(ls) 
loss = Loss()
EPOCHS= 100

ds = Dataset(testing=True) # takes 160 examples to overfit model too for testing purposes
splits = ['train','val','test']
print(f"dataset sizes for {', '.join(splits)} are {[ds.__len__(split) for split in splits]} respectively.")
dst = ds.train
dsv = ds.val
dss = ds.test
# iterators 
dltrain  = DataLoader(dst,32)
dlval = DataLoader(dsv,32)
dltest = DataLoader(dss,23)


EARLY_STOPPING = None
losses = []
for epoch in range(EPOCHS):
	print("Training".center(70,"#"))
	for (idx,xy) in enumerate( dltrain()):
		x,y = xy 
		model = model.zero_grad()
		# print("After zero grad")
		# print(model.__dict__)

		y_hat =model(x) 
		
		acc, avg_loss = loss(y,y_hat)

		avg_loss.backward()

		losses.append(avg_loss.data)
		model = model.step()
		print(f'Epoch {epoch}:{idx} Loss:{avg_loss.data} train accuracy:{acc*100}%')
			
		# lr = 0.00000005 
		# for p in model.parameters():
		# 	#print('p.data:', p.data,  'p.grad:',p.grad)			
		# 	p.data = p.data -  lr * p.grad
        
		# model.zero_grad()

		# # checking if gradients were actually set to zero 
		# for p in model.parameters():
		# 	print('p.grad:',p.grad)

	print("Validation".center(70,"#"))
	for (idx,xy) in enumerate( dlval()):
		x,y = xy 
#			for layer in model.layers:
#				x = layer(x)
		y_hat = model(x)
		acc, avg_loss= loss(y,y_hat)
		print(f'Epoch {epoch}:{idx} Loss:{avg_loss.data} val accuracy:{acc*100}%')	

# train()

layer init seed: 6158578929613097487
dataset seed: 6320435684609269161
dataset sizes for train, val, test are [112, 32, 16] respectively.
###############################Training###############################
Epoch 0:0 Loss:1.3673959157079156 train accuracy:56.25%
Epoch 0:1 Loss:1.554634849275223 train accuracy:53.125%
Epoch 0:2 Loss:1.2364061430681685 train accuracy:53.125%


In [ ]:
names = ["l1","l2","l3","l4", "l5"]		
layers =[l1,l2,l3,l4, l5]
ls = dict(zip(names,layers))

model = Model(ls) 

model.__dict__


In [ ]:
!pip install ipympl

In [ ]:
%matplotlib widget
%matplotlib inline


In [ ]:
from dydx.dataset import DataLoader, Dataset
ds = Dataset(testing=True) # takes 160 examples to overfit model too for testing purposes
splits = ['train','val','test']
print(f"dataset sizes for {', '.join(splits)} are {[ds.__len__(split) for split in splits]} respectively.")
dst = ds.train
dsv = ds.val
dss = ds.test
# iterators 
dltrain  = DataLoader(dst,32)
dlval = DataLoader(dsv,32)
dltest = DataLoader(dss,23)


import torch.nn as nn
from torch.optim import SGD
import torch.nn.functional as F
from torch import Tensor


# PyTorch models inherit from torch.nn.Module
class ClaimsClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 32)
        self.fc5 = nn.Linear(32,1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.sigmoid(x)
    

model = ClaimsClassifier()

for _ in range(100000):
    optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
    for xy in dltrain():
        x, y = xy
        x = Tensor([[x_.data for x_ in row] for row in x.values])
        y = Tensor([y_.data for y_ in y.values])
        
        #print(Tensor(x))
        #print(Tensor(y))

        optimizer.zero_grad()
        y_hat = model(x)
        y_hat = y_hat.squeeze(1)
        #print(y_hat.shape)
        #print(y_hat)
        loss= F.binary_cross_entropy(y_hat,y)
        loss.backward()
        optimizer.step()
        print(loss)



    


